In [97]:
# import libraries.
import requests
import json
import os
import os
import asyncio
import aiohttp

In [119]:
# get real time data from backend elastic search

# server config, using portforwarding
def get_data(restpath, filename):
    BASE_URL = "http://127.0.0.1:9090"
    URL = f"{BASE_URL}/{restpath}"
    print(URL)
    try:
         with requests.get(URL, stream=True) as response:
            if response.status_code == 200:
                print(f"Retrieving data from {URL}...")
                data_list = []
                for line in response.iter_lines():
                    if line:
                        data = json.loads(line.decode('utf-8'))
                        data_list.append(data)

                # 异步保存数据到文件
                save_data_to_file(data_list, filename)
    except requests.exceptions.RequestException as e:
        print(f"Error: An exception occurred while requesting data from {URL}. Exception: {e}")
        return None
    
def save_data_to_file(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to {filename}")


In [184]:
# Obtain data from the backend server
ROUTE_LIST = ["aircondition","weathercondition","mastodon-aus-social"]
data_folder = './realtimeData'
for route in ROUTE_LIST:
        restpath = "database/" + route
        filename = os.path.join(data_folder, f"{route}.json")
        data = get_data(restpath,filename)




http://127.0.0.1:9090/datatype/aircondition


http://127.0.0.1:9090/datatype/weathercondition
http://127.0.0.1:9090/datatype/mastodon-aus-social


In [181]:
data = get_data("database/realestate-affordability", "realestate-affordability.json")
print('finish')

http://127.0.0.1:9090/datatype/realestate-affordability
Retrieving data from http://127.0.0.1:9090/datatype/realestate-affordability...
Error: An exception occurred while requesting data from http://127.0.0.1:9090/datatype/realestate-affordability. Exception: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
finish


In [185]:
# Obtain data from different time period
route_name = "database/mastodon-aus-social/timeperiod/2024-05-19-10/2024-05-19-13"
data = get_data(route_name, "timeperiod.json")
print(data)

http://127.0.0.1:9090/datatype/mastodon-aus-social/timeperiod/2024-05-19-10/2024-05-19-13


KeyboardInterrupt: 